In [ ]:
import json
import random
import pickle
import numpy as np 

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
with open('dialogflowintents.json') as file:
    data = json.load(file)

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
training_sentences #parallel with labels

In [ ]:
training_labels #parallel with sentences

In [ ]:
responses

In [ ]:
#creating features
Xfeatures = training_sentences #need to be vectorized
ylabels = training_labels #can remain as it is

In [ ]:
cv = CountVectorizer()

In [ ]:
X = cv.fit_transform(Xfeatures)

In [ ]:
nv_model = MultinomialNB()
nv_model.fit(X, ylabels)

MultinomialNB()

In [ ]:
def reply(tag):
  for i in data['intents']:
    if i['tag'] == tag:
      print("TheraBot:", np.random.choice(i['responses']))

In [ ]:
emotion_scores = {
    "angry" : 0,
    "farewell" : 0, 
    "fear" : 0, 
    "greetings" : 0,
    "joy" : 0, 
    "sad" : 0,
    "thanks" : 0
}

def predict_emotion_score(sample_text, model):
    myvect = cv.transform(sample_text).toarray()
    prediction = model.predict(myvect)
    pred_proba = model.predict_proba(myvect)
    pred_percentage_for_all = dict(zip(model.classes_, pred_proba[0]))
    print("Prediction: {}, Prediction Score: {}".format(prediction[0], np.max(pred_proba)))
    emotion_scores['angry'] += pred_percentage_for_all['angry']
    emotion_scores['farewell'] += pred_percentage_for_all['farewell']
    emotion_scores['fear'] += pred_percentage_for_all['fear']
    emotion_scores['greetings'] += pred_percentage_for_all['greetings']
    emotion_scores['joy'] += pred_percentage_for_all['joy']
    emotion_scores['sad'] += pred_percentage_for_all['sad']
    emotion_scores['thanks'] += pred_percentage_for_all['thanks']
    reply(prediction[0])
    return pred_percentage_for_all

In [ ]:
def get_highest_key():
  highest_value = 0
  for key in emotion_scores.keys():
    if emotion_scores[key] >= highest_value:
      highest_value = emotion_scores[key]
      highest_key = key
  return highest_key

def consolidation_message(highest_key):
    if highest_key == 'sad':
      print("ChatBot: You seem to be really sad, maybe consult a therapist or talk to a close friend")
    elif highest_key == 'joy':
      print("You seem to be in a good mood, hope you feel this way always!")
    elif highest_key == 'angry':
      print("You seem agitated right now, take a deep breath")
    elif highest_key == 'fear':
      print("I understand you feel scared, this too shall pass.")
    else:
      print("Goodbye! Take care")

def chat():
    while True:
        print("User: ", end="")
        user_message = input()
        if user_message.lower() == "quit":
            highest_key = get_highest_key()
            consolidation_message(highest_key)
            break
        else:
          predict_emotion_score([user_message], nv_model)

print("Start messaging with the bot. Type 'quit' to end session. This conversation remains confidential.")
chat()

Start messaging with the bot. Type 'quit' to end session. This conversation remains confidential.
User: im happy
Prediction: joy, Prediction Score: 0.3565995006959957
TheraBot: You are happy
User: im sad
Prediction: sad, Prediction Score: 0.3180923260755737
TheraBot: You are sad
User: i dont feel good
Prediction: sad, Prediction Score: 0.5675767283488105
TheraBot: You are sad
User: i feel good
Prediction: sad, Prediction Score: 0.4390749661991964
TheraBot: You are sad
User: quit
ChatBot: You seem to be really sad, maybe consult a therapist or talk to a close friend
